In [ ]:
!pip install geopandas fiona 

     |████████████████████████████████| 972kB 20.8MB/s 
     |████████████████████████████████| 14.8MB 216kB/s 
     |████████████████████████████████| 6.5MB 49.5MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Goal here is to read in & access this ~1gb file
#File is in this area (of my drive, at least): /content/drive/My Drive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_grip4
# Trying: https://gis.stackexchange.com/questions/32762/how-to-access-feature-classes-in-file-geodatabases-with-python-and-gdal (look far down)

#Data taken from: https://www.globio.info/download-grip-dataset
#Some similar data here: https://enterprisecontentnew-usfs.hub.arcgis.com/datasets/motor-vehicle-use-map-roads-feature-layer?page=17611


import fiona 
import geopandas as gpd

# Get all the layers from the .gdb file 
#%%%%%%%%%%%%%%%%% We May Have Different Drive Paths To This Folder %%%%%%%%%%%%%%%%%%%%
gdb_file_path = '/content/drive/MyDrive/ENGS108_Final_Project/Data_Sets/GRIP4_region1.gdb'

water_bodies_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/hydropolys.gdb'
linear_water_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/hydrolines.gdb'

layers = fiona.listlayers(gdb_file_path)


for layername in fiona.listlayers(gdb_file_path):
    with fiona.open(gdb_file_path, layer=layername) as src:
        print(layername, len(src))
c = fiona.open(gdb_file_path, 'r')
print("A sample road vector, representing 1 road:")
print(c[10060])

linear_water = fiona.open(linear_water_path, 'r')
poly_water = fiona.open(water_bodies_path, 'r')

GRIP4_region1 5360526
A sample road vector, representing 1 road:
{'type': 'Feature', 'id': '10060', 'properties': OrderedDict([('GP_RTP', 5), ('GP_REX', 1), ('GP_RAV', 2), ('GP_RRG', 1), ('GP_RCY', 124), ('GP_RSE', 0), ('GP_RSI', 54), ('GP_RSY', 2014), ('gp_gripreg', 1), ('Shape_Length', 0.001259744524104104)]), 'geometry': {'type': 'MultiLineString', 'coordinates': [[(-113.5190227, 53.607822000000056), (-113.51784859999998, 53.60790960000003), (-113.51777529999998, 53.60794720000007)]]}}


In [ ]:
print("A sample road vector, representing 1 road:")
c[6]['geometry']['coordinates']

A sample road vector, representing 1 road:


[[(-123.15683999999999, 49.15667000000008),
  (-123.15675999999996, 49.14998000000003),
  (-123.15675999999996, 49.14206000000007),
  (-123.15674999999999, 49.134930000000054)]]

In [36]:
import datetime
now = datetime.datetime.now()
print(now)

'''
Finds the indeces of c that are within our desired range
'''
def linear_grid_finder(clist,lat_min,lat_max,long_min,long_max, kind):
  idcs = []
  for i in range(1,len(clist)+1):
      if kind!=1:
        if lat_min < clist[i]['geometry']['coordinates'][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0] < long_max:
            idcs.append(i)
      else:
         if lat_min < clist[i]['geometry']['coordinates'][0][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0][0] < long_max:
            idcs.append(i)
  return idcs

now = datetime.datetime.now()
print(now)

2020-12-02 19:34:13.944813
2020-12-02 19:34:13.945119


In [ ]:
'''
Splits a rectangular surface into smaller rectangles and returns them all
'''
def split_grid(num_rows,num_cols,x_min,x_max,y_min,y_max):
    quad_list = []
    increment_rows = (y_max - y_min)/num_rows
    increment_cols = (x_max - x_min)/num_cols

    for i in range(1,num_rows):
      for j in range(1,num_cols):
          quad_list.append((x_min+(i-1)*increment_cols, x_min+(i)*increment_cols, y_min+(j-1)*increment_rows, y_min+(j)*increment_rows))
    return quad_list

In [ ]:
print(split_grid(10,10,0,100,0,100))

[(0.0, 10.0, 0.0, 10.0), (0.0, 10.0, 10.0, 20.0), (0.0, 10.0, 20.0, 30.0), (0.0, 10.0, 30.0, 40.0), (0.0, 10.0, 40.0, 50.0), (0.0, 10.0, 50.0, 60.0), (0.0, 10.0, 60.0, 70.0), (0.0, 10.0, 70.0, 80.0), (0.0, 10.0, 80.0, 90.0), (10.0, 20.0, 0.0, 10.0), (10.0, 20.0, 10.0, 20.0), (10.0, 20.0, 20.0, 30.0), (10.0, 20.0, 30.0, 40.0), (10.0, 20.0, 40.0, 50.0), (10.0, 20.0, 50.0, 60.0), (10.0, 20.0, 60.0, 70.0), (10.0, 20.0, 70.0, 80.0), (10.0, 20.0, 80.0, 90.0), (20.0, 30.0, 0.0, 10.0), (20.0, 30.0, 10.0, 20.0), (20.0, 30.0, 20.0, 30.0), (20.0, 30.0, 30.0, 40.0), (20.0, 30.0, 40.0, 50.0), (20.0, 30.0, 50.0, 60.0), (20.0, 30.0, 60.0, 70.0), (20.0, 30.0, 70.0, 80.0), (20.0, 30.0, 80.0, 90.0), (30.0, 40.0, 0.0, 10.0), (30.0, 40.0, 10.0, 20.0), (30.0, 40.0, 20.0, 30.0), (30.0, 40.0, 30.0, 40.0), (30.0, 40.0, 40.0, 50.0), (30.0, 40.0, 50.0, 60.0), (30.0, 40.0, 60.0, 70.0), (30.0, 40.0, 70.0, 80.0), (30.0, 40.0, 80.0, 90.0), (40.0, 50.0, 0.0, 10.0), (40.0, 50.0, 10.0, 20.0), (40.0, 50.0, 20.0, 30.0),

In [ ]:
def return_coords(clist, idx, kind):
  if kind == 1:
     return clist[idx]['geometry']['coordinates'][0][0]
  else:
     return clist[idx]['geometry']['coordinates'][0]

def return_all_coords(clist, idcs, kind=0):
  all_coords=[]
  for idx in idcs:
      if clist[idx]:
        if len(clist[idx]['geometry']['coordinates'])==1:
          all_coords.append(return_coords(clist,idx, kind))
        else:
          for i in range(len(clist[idx]['geometry']['coordinates'])):
              all_coords.append(clist[idx]['geometry']['coordinates'][i])
  return all_coords

In [ ]:
import matplotlib.pyplot as plt

def prepare_plot(roads_data):
  prepared_dataX = []
  prepared_dataY = []
  for feature in roads_data:
    these_xs = []
    these_ys = []
    try:
      for x, y in feature:
        these_xs.append(x)
        these_ys.append(y)
    except:
      for x, y in feature[0]:
        these_xs.append(x)
        these_ys.append(y)

    prepared_dataX.append(these_xs)

    prepared_dataY.append(these_ys)
  return prepared_dataX, prepared_dataY


In [38]:
from matplotlib.pyplot import figure
plt.ioff()
import numpy as np

def create_image(roadX, roadY, waterX_poly, waterY_poly, waterX_lin, waterY_lin,n):
  fig = figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k') #Resize to 1000x1000 images
  if roadX:
    for i in range(0, len(roadX)):
      tuple_setX = roadX[i]
      tuple_setY = roadY[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 2, color = 'grey')
  if waterX_lin:
    for i in range(0, len(waterX_lin)):
      tuple_setX = waterX_lin[i]
      tuple_setY = waterY_lin[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 8, color = 'blue')
  if waterX_poly:
      for i in range(len(waterX_poly)):
          plt.fill(waterX_poly[i], waterY_poly[i], edgecolor='blue', facecolor='blue')
  
      
  plt.axis('off')
  plt.savefig('/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/roads_only/roads_' +  str(n) + '.png', bbox_inches = 'tight')

In [39]:
total=1000

In [ ]:
goal_lat_min = 40.0
goal_lat_max = 60.0
x_cuts = int(abs(goal_lat_max-goal_lat_min)*20)
goal_long_min = -80.0
goal_long_max = -60.0
y_cuts = int(abs(goal_long_max-goal_long_min)*20)
list_of_coords = split_grid(x_cuts,y_cuts,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max)
for k in range(7,len(list_of_coords)):
    min_lat, max_lat, min_long, max_long = list_of_coords[k]
    print('entered for')
    idcs = linear_grid_finder(c, min_lat, max_lat, min_long, max_long, 0)
    if len(idcs) > 5:
      total+=1
      print(k)
      all_coords = return_all_coords(c,idcs)
      #idcs = linear_grid_finder(poly_water, min_lat, max_lat, min_long, max_long, 1)
      #lake_coords = return_all_coords(poly_water, idcs,1)
      #idcs = linear_grid_finder(linear_water,  min_lat, max_lat, min_long, max_long, 2)
      #river_coords = return_all_coords(linear_water, idcs,2)
      roadX,roadY = prepare_plot(all_coords)
      #wpolyX, wpolyY = prepare_plot(lake_coords)
      #wlinX, wlinY = prepare_plot(river_coords)
      create_image(roadX,roadY, None, None, None, None,k+total)

entered for
7
entered for
8
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
17
entered for
18
entered for
19
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
29
entered for
30
entered for
entered for
32
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
47
entered for
48
entered for
49
entered for
50
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
59
entered for
60
entered for
entered for
62
entered for
63
entered for
64
entered for
65
entered for
66
entered for
67


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


entered for
68
entered for
69
entered for
70
entered for
71
entered for
72
entered for
73
entered for
74
entered for
75
entered for
76
entered for
77
entered for
78
entered for
entered for
entered for
81
entered for
82
entered for
83
entered for
84
entered for
85
entered for
86
entered for
87
entered for
88
entered for
89
entered for
90
entered for
91
entered for
92
entered for
93
entered for
94
entered for
95
entered for
96
entered for
97
entered for
98
entered for
99
entered for
100
entered for
101
entered for
102
entered for
103
entered for
104
entered for
entered for
entered for
107
entered for
108
entered for
entered for
entered for
entered for
entered for
113
entered for
114
entered for
115
entered for
116
entered for
117
entered for
118
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entered for
entere